![car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

Identify the single feature of the data that is the best predictor of whether a customer will put in a claim (the "outcome" column), excluding the "id" column.
Store as a DataFrame called best_feature_df, containing columns named "best_feature" and "best_accuracy" with the name of the feature with the highest accuracy, and the respective accuracy score.

In [93]:
# Import required modules
import pandas as pd
import numpy as np
#from statsmodels.formula.api import logit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## EDA & Cleaning

In [94]:
# import data
insurance = pd.read_csv('car_insurance.csv')

insurance.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0-9y,high school,upper class,0.629027,1.0,after 2015,0.0,1.0,10238,12000.0,sedan,0,0,0,0.0
1,750365,0,1,0-9y,none,poverty,0.357757,0.0,before 2015,0.0,0.0,10238,16000.0,sedan,0,0,0,1.0
2,199901,0,0,0-9y,high school,working class,0.493146,1.0,before 2015,0.0,0.0,10238,11000.0,sedan,0,0,0,0.0
3,478866,0,1,0-9y,university,working class,0.206013,1.0,before 2015,0.0,1.0,32765,11000.0,sedan,0,0,0,0.0
4,731664,1,1,10-19y,none,working class,0.388366,1.0,before 2015,0.0,0.0,32765,12000.0,sedan,2,0,1,1.0


In [95]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

In [96]:
# null values
insurance.isna().sum()

# describe colums with missing data
insurance[['credit_score', 'annual_mileage']].describe()

# impute average values respectively
insurance['credit_score'] = insurance['credit_score'].fillna(insurance['credit_score'].mean())
insurance['annual_mileage'] = insurance['annual_mileage'].fillna(insurance['annual_mileage'].median())

In [97]:
# clean values in 'driving_experience'
insurance['driving_experience'] = insurance['driving_experience'].replace({'0-9y': '0',
                                                                           '10-19y': '1',
                                                                           '20-29y': '2',
                                                                           '30y+': '3'})

# convert to int
insurance['driving_experience'] = insurance['driving_experience'].astype('int32')

In [98]:
# clean values in 'education'
insurance['education'] = insurance['education'].replace({'none': '0',
                                                         'high school': '1',
                                                         'university': '2'})

# convert to int
insurance['education'] = insurance['education'].astype('int32')

In [99]:
# clean values in 'income'
insurance['income'] = insurance['income'].replace({'poverty': '0',
                                                   'working class': '1',
                                                   'middle class': '2',
                                                   'upper class': '3'})

# convert to int
insurance['income'] = insurance['income'].astype('int32')

In [100]:
# clean values in 'vehicle_year'
insurance['vehicle_year'] = insurance['vehicle_year'].replace({'before 2015': '0',
                                                               'after 2015': '1'})

# convert to int
insurance['vehicle_year'] = insurance['vehicle_year'].astype('int32')

In [101]:
# clean values in 'postal_code'
insurance['postal_code'] = insurance['postal_code'].replace({'10238': '0',
                                                             '32765': '1',
                                                             '92101': '2',
                                                             '21217': '3'})

# convert to int
insurance['postal_code'] = insurance['postal_code'].astype('int32')

In [102]:
# clean values in 'vehicle_type'
insurance['vehicle_type'] = insurance['vehicle_type'].replace({'sedan': '0',
                                                              'sports car': '1'})

# convert to int
insurance['vehicle_type'] = insurance['vehicle_type'].astype('int32')

In [103]:
# convert other columns to int
insurance['vehicle_ownership'] = insurance['vehicle_ownership'].astype('int32')
insurance['married'] = insurance['married'].astype('int32')
insurance['children'] = insurance['children'].astype('int32')
insurance['annual_mileage'] = insurance['annual_mileage'].astype('int32')
insurance['outcome'] = insurance['outcome'].astype('int32')

In [104]:
insurance.head()

,id,age,gender,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
0,569520,3,0,0,1,3,0.629027,1,1,0,1,10238,12000,0,0,0,0,0
1,750365,0,1,0,0,0,0.357757,0,0,0,0,10238,16000,0,0,0,0,1
2,199901,0,0,0,1,1,0.493146,1,0,0,0,10238,11000,0,0,0,0,0
3,478866,0,1,0,2,1,0.206013,1,0,0,1,32765,11000,0,0,0,0,0
4,731664,1,1,1,0,1,0.388366,1,0,0,0,32765,12000,0,2,0,1,1


In [105]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  int32  
 4   education            10000 non-null  int32  
 5   income               10000 non-null  int32  
 6   credit_score         10000 non-null  float64
 7   vehicle_ownership    10000 non-null  int32  
 8   vehicle_year         10000 non-null  int32  
 9   married              10000 non-null  int32  
 10  children             10000 non-null  int32  
 11  postal_code          10000 non-null  int32  
 12  annual_mileage       10000 non-null  int32  
 13  vehicle_type         10000 non-null  int32  
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

## Logistic Regression Model

In [106]:
# features and targets
features = insurance.drop(['id', 'outcome'], axis = 1)
y = insurance['outcome']

# list for our accuracies
feature_pred_acc = []

# instantiate and fit model
model = LogisticRegression()

In [107]:
# iterate over feature columns and create a model for each
for column in features.columns:
    
    # select column
    X = features[[column]]
    
    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    #fit model
    model.fit(X_train, y_train)
    
    # predictions
    y_pred = model.predict(X_test)
    
    # accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    feature_pred_acc.append(accuracy)

In [108]:
# dict of features and prediction accuracies
feature_accuracies = {'feature': features.columns,
                      'accuracy': feature_pred_acc}

# feature and accuracy df
feature_accuracies_df = pd.DataFrame(feature_accuracies).sort_values(by = 'accuracy', ascending = False)

feature_accuracies_df

,feature,accuracy
0,age,0.7830
2,driving_experience,0.7780
4,income,0.7440
6,vehicle_ownership,0.7280
5,credit_score,0.6990
1,gender,0.6835
3,education,0.6835
7,vehicle_year,0.6835
8,married,0.6835
9,children,0.6835


In [109]:
# best_feature_df
best_feature_df = pd.DataFrame({'best_feature': feature_accuracies_df['feature'].iloc[0],
                                'best_accuracy': feature_accuracies_df['accuracy'].iloc[0]},
                                index = [0])

best_feature_df

,best_feature,best_accuracy
0,age,0.783
